In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import os
from statsmodels.tsa.arima_model import ARIMA

In [9]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [10]:
def formatFilename(filename):
    return filename.replace(".csv", "")

In [11]:
test_index = 4 #Use first year to train, second to test
for (p,d,q) in [(p,d,q) for p in range(5) for d in range(2) for q in range(5)]:
    try:
        for country in ["Mexico", "Brazil", "Colombia"]:
            print(country)
            print(p,d,q)
            print("---")
            folder = "../../data/{}/processed_data".format(country)
            files = os.listdir(folder)


            for file in files:

                outFolder = "{},{},{}".format(p,d,q)
                if(not os.path.exists("{}/{}/{}".format(country, outFolder, file))):
                    print("Performing experiment in {}/{}/{}".format(country, outFolder, file))
                    dataset = pd.read_csv("{}/{}".format(folder, file))
                    formattedFilename = formatFilename(file)
                    #Preprocess data
                    dataset["Searches"] /= 100
                    state = file
                    dataset["Cases"] *= (100000/population[state]) # Transform to incidence per 100,000 habitants    

                    #Develop Sarimax model
                    model= ARIMA(dataset["Cases"].values,
                                 order=(p,d,q), freq="W")
                    #Fit model
                    results = model.fit()
                #     print(results.summary())

                    #Predict for next year
                    predictions = results.predict(
                    start = test_index,             
#                     dynamic= False, #lagged values are used in next prediction
# #                     exog=dataset["Searches"][test_index:].values.reshape(52,1)
                    )

                    #Store results
                    predictions_df = pd.DataFrame()
                    predictions_df["predictions_incidence"] = predictions
                    predictions_df["observed_incidence"] = dataset["Cases"][test_index:].values

                    predictions_df["Predicted Cases"] = predictions * (population[state] / 100000)
                    predictions_df["Observed Cases"] = dataset["Cases"][test_index:].values * (population[state] / 100000)

                    predictions_df["error"] = predictions_df["Predicted Cases"] - predictions_df["Observed Cases"]
                    predictions_df.set_index(dataset[test_index:]["Date"], inplace=True)

                    outFolder = "{},{},{}".format(p,d,q)

                    if(not os.path.isdir("{}/{}".format(country, outFolder))):
                        os.mkdir("{}/{}".format(country, outFolder))

                    if(not os.path.isdir("{}/{}/{}".format(country, outFolder,file))):
                        os.mkdir("{}/{}/{}".format(country, outFolder, file))

                    predictions_df.to_csv("{}/{}/{}/Arimax-{}.csv".format(country, outFolder, file, formattedFilename))

                    #Save images
                    plt.clf()

                    cols = ['Observed Cases', 'Predicted Cases']
                    colors = ['#2962FF', '#FF9800']

                    predictions_df[cols].plot(
                        figsize=(10, 10),
                        grid=True,
                        color=colors
                    )

                    plt.title("Arimax Model\n{}".format(formattedFilename.replace("_2016-"," ")))
                    ax = plt.gca()
                    ax.set_facecolor((0.9, 0.9, 0.9, 0.7))
                    plt.xlabel("Date")
                    plt.ylabel("ZIKV Cases")
                    plt.grid(linestyle='dashed', linewidth=1.5)
                    fig = plt.gcf()
                    fig.savefig("{}/{}/{}/Arimax-{}.png".format(country, outFolder, file, formattedFilename))
                    plt.close("all")
                else:
                    print("Experiment already exists")
    except Exception as e:
        print(e)

Mexico
0 0 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
0 0 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
0 0 2
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already 

E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:584: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:585: RuntimeWarning: overflow encountered in exp
  (1+np.exp(-params))).copy()
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:585: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:586: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:587: RuntimeWarning: overflow encountered in exp
  (1+np.exp(-params))).copy()
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:587: RuntimeWarning: invalid value encountered in true_divide
  (1+np.exp(-params))).copy()


SVD did not converge
Mexico
2 0 3
---
Performing experiment in Mexico/2,0,3/Chiapas_2016-2017.csv
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
Mexico
2 0 4
---
Performing experiment in Mexico/2,0,4/Chiapas_2016-2017.csv
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
Mexico
2 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
2 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
2 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
2 1 1
---
Experiment already exists
Experiment already ex

4 1 2
---
Experiment already exists
Experiment already exists
Experiment already exists
Performing experiment in Colombia/4,1,2/tolima_2016-2017.csv
SVD did not converge
Mexico
4 1 3
---
Experiment already exists
Experiment already exists
Experiment already exists
Performing experiment in Mexico/4,1,3/Veracruz_2016-2017.csv
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
Mexico
4 1 4
---
Experiment already exists
Experiment already exists
Experiment already exists
Performing experiment in Mexico/4,1,4/Veracruz_2016-2017.csv
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:654: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:628: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:629: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
